In [18]:
import pandas as pd

In [19]:
df = pd.read_csv('/content/sales_transactions.csv')
df.head()

,Transaction_ID,Customer_ID,Region,Product_Category,Product_Name,Quantity,Unit_Price ($),Total_Sales ($),Transaction_Date,Payment_Method
0,10001,2861,East,Books,Biography,3,155.58,466.74,2024-09-09,PayPal
1,10002,3295,West,Toys,Action Figure,8,439.08,3512.64,2023-09-14,PayPal
2,10003,3131,North,Toys,Doll,8,276.15,2209.20,2024-08-17,Credit Card
3,10004,3096,North,Electronics,Laptop,2,291.73,583.46,2025-01-04,PayPal
4,10005,3639,South,Beauty,Perfume,7,332.55,2327.85,2024-03-09,PayPal


In [20]:
# checking for null values
df.isna().sum() #no missing values
df.duplicated().sum() # No duplicate value

# standardizing column names
# first convert all to lower case, then replace $ with _USD and replace every
# with underscore
df.columns = df.columns.str.lower().str.replace(' ($)', '_(usd)').str.replace(' ', '_')

# checking the basic info of the df
df.describe()

# converting the date column to proper datetime type
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   transaction_id     10000 non-null  int64         
 1   customer_id        10000 non-null  int64         
 2   region             10000 non-null  object        
 3   product_category   10000 non-null  object        
 4   product_name       10000 non-null  object        
 5   quantity           10000 non-null  int64         
 6   unit_price_(usd)   10000 non-null  float64       
 7   total_sales_(usd)  10000 non-null  float64       
 8   transaction_date   10000 non-null  datetime64[ns]
 9   payment_method     10000 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 781.4+ KB


# 1️⃣ Basic Summary Statistics

In [22]:
# Calculate the total revenue across all transactions.
total_revenue = df['total_sales_(usd)'].sum()
print(f"The total revenue obtained from all transactions was ${total_revenue:.0f}")

# Find the average transaction value (Total_Sales).
avg_txn_value = df['total_sales_(usd)'].mean()
print(f"The average transaction value was ${avg_txn_value:.2f}")

# Get the number of unique customers.
unique_customers = df['customer_id'].nunique()
print(f"There were {unique_customers} unique customers")

# Find the highest and lowest Total_Sales values.
max_total_sales = df['total_sales_(usd)'].max()
min_total_sales = df['total_sales_(usd)'].min()
print(f"The maximum total sales was ${max_total_sales:.2f} and the minimum total sales was ${min_total_sales:.2f}")

# Compute the average number of products purchased per transaction.
avg_quantity = df['quantity'].mean()
print(f"The average quantity of items purchased per transaction was {avg_quantity:.2f}")

The total revenue obtained from all transactions was $13997042
The average transaction value was $1399.70
There were 2889 unique customers
The maximum total sales was $4990.90 and the minimum total sales was $5.33
The average quantity of items purchased per transaction was 5.53


# 2️⃣ Aggregation Using `.groupby()`

In [26]:
# Find total revenue per Region.
revenue_by_region = df.groupby('region')['total_sales_(usd)'].agg('sum').reset_index()
#sorting the values in descending order
revenue_by_region.sort_values(
    by='total_sales_(usd)', ascending=False, inplace=True
    )
#reseting the column names of the new table
revenue_by_region.columns = ['Region', 'Total Revenue']
revenue_by_region


# Compute average Total_Sales per Product_Category.
avg_sales_per_category = df.groupby('product_category')['total_sales_(usd)'].agg('mean').reset_index()
avg_sales_per_category.columns = ['Product Category', 'Average Sales'] #renaming columns
avg_sales_per_category

txns_per_payment_method = df.groupby('payment_method')['transaction_id'].agg('count').reset_index()
txns_per_payment_method.columns = ['Payment Method', 'Number of Transactions']
# reorder the table
txns_per_payment_method.sort_values(by = 'Number of Transactions', inplace = True)
txns_per_payment_method

prod_sales_by_reg = df.groupby(['region', 'product_name'])['total_sales_(usd)'].agg('sum').reset_index()
max_sales_by_reg = prod_sales_by_reg.loc[prod_sales_by_reg.groupby('region')['total_sales_(usd)'].idxmax()]
max_sales_by_reg

,region,product_name,total_sales_(usd)
22,East,Tablet,162335.31
25,North,Biography,189718.57
56,South,Foundation,177236.21
92,West,Sneakers,221582.62


In [31]:
for index, row in max_sales_by_reg.iterrows():
  print(f"{row['product_name']} is the highest selling product in the {row['region']}")

Tablet is the highest selling product in the East
Biography is the highest selling product in the North
Foundation is the highest selling product in the South
Sneakers is the highest selling product in the West


In [32]:
total_customer_sales = df.groupby('customer_id')['total_sales_(usd)'].sum().reset_index()
total_customer_sales.columns = ['Customer ID', 'Total Sales']

,Customer ID,Total Sales
0,2001,255.86
1,2002,8164.15
2,2003,1260.19
3,2004,3755.70
4,2005,372.10
...,...,...
2884,4995,4672.10
2885,4996,6069.52
2886,4997,2459.72
2887,4998,88.55


# 3️⃣ Multi-Level Aggregation